In [1]:
from ethpandaops_python.hypersync import Hypersync
from ethpandaops_python.preprocessor import Preprocessor
from holoviews import opts
import nest_asyncio
import polars as pl
import panel as pn


nest_asyncio.apply()
pn.extension("plotly", template="material", sizing_mode="stretch_width")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Data Query

In [2]:
# labeled blobs - https://dune.com/queries/3521610
arbitrum: str = "0xC1b634853Cb333D3aD8663715b08f41A3Aec47cc"
linea: str = "0xa9268341831eFa4937537bc3e9EB36DbecE83C7e"
base: str = "0x5050F69a9786F081509234F1a7F4684b5E5b76C9"
starknet: str = "0x2C169DFe5fBbA12957Bdd0Ba47d9CEDbFE260CA7"
num_days: int = 5
preprocessor: Preprocessor = Preprocessor(
    blob_producer=base,
    period=num_days,
    network="mainnet",  # mainnet
)

client = Hypersync()

txs_df = client.query_txs(address=base.lower(), period=(num_days))

In [3]:
# get preprocessed data
slot_inclusion_df: pl.DataFrame = preprocessor.slot_inclusion()

In [4]:
processed_slot_inclusion_df = (
    slot_inclusion_df.drop_nulls()
    .drop(
        "meta_network_name",
        "block_root",
        "blob_size",
        "from",
        "to",
        "blob_hashes_length",
        "blob_sidecars_size",
    )
    # there is an outlier in the data that has an unusually high slot count, I think it might be an error with the xatu mempool data.
    # additionally there is currently a single tx with 5 blobs and 1 with 7. Since it's a single row, I think it's easier to just discard and ignore for now
    .filter(pl.col("slot inclusion rate") < 250)
    .filter(pl.col("slot inclusion rate (50 blob average)") < 40)
)

In [5]:
joined_processed_slot_inclusion_df = processed_slot_inclusion_df.join(
    txs_df, on="hash", how="left"
)

In [6]:
slot_inclusion_tx_df = (
    (
        joined_processed_slot_inclusion_df.with_columns(
            (pl.col("effective_gas_price") / 10**9)
            .round(3)
            .alias(
                "effective_gas_price_gwei"
            ),  # gas price in gwei that was paid, including priority fee
            (pl.col("max_fee_per_gas") / 10**9)
            .round(3)
            .alias(
                "max_fee_per_gas_gwei"
            ),  # max gas price in gwei that rollup is willing to pay
            (pl.col("max_priority_fee_per_gas") / 10**9)
            .round(3)
            # priority gas fee in gwei,
            .alias("max_priority_fee_per_gas_gwei"),
        )
    )
    .drop_nulls()
    .sort(by="submission_count", descending=True)
)

In [7]:
slot_inclusion_tx_df.head(5)

versioned_hash,nonce,event_date_time_min,event_date_time_max,fill_percentage,blob_gas,blob_gas_fee_cap,gas_price,gas_tip_cap,gas_fee_cap,hash,submission_count,slot,slot time,kzg_commitment,blob_empty_size,beacon_inclusion_time,slot inclusion rate,slot inclusion rate (50 blob average),slot target inclusion rate (2 slots),block_number,from_,to,gas,transaction_index,gas_price_right,effective_gas_price,gas_used,cumulative_gas_used,max_fee_per_gas,max_priority_fee_per_gas,effective_gas_price_gwei,max_fee_per_gas_gwei,max_priority_fee_per_gas_gwei
str,u64,datetime[ms],datetime[ms],f64,f64,f64,f64,f64,f64,str,u32,u32,datetime[ms],str,u32,f64,f64,f64,i32,i64,str,str,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0x01a9ecb35e0e9910dc87e6c0968732d981837ae18bd5b224b89d98143eb8067e""",630605,2024-04-07 19:30:43.471,2024-04-07 19:30:43.624,100,131072,150000000000,150000000000,100000000,150000000000,"""0x0cb9f511b65649a769b448e2f141490720bd7df40f8ca2e0292e86ca6e113014""",1,8807852,2024-04-07 19:30:47,"""0xa46aa097a070072bb28133649cab1137a1db81704f6d10a13a1c64f457b0a9e092cee5a0f6f6e275775c1f4e5761fbf6""",0,3.529,1,2.4,2,19605908,"""0x2c169dfe5fbba12957bdd0ba47d9cedbfe260ca7""","""0xc662c410c0ecf747543f5ba90660f6abebd9c8c4""",5500000,69,16472849688,16472849688,136612,6389886,150000000000,100000000,16.473,150,0.1
"""0x0117b71cad467a9633fcf54f9e8bd9acba083600f823f9538a15d3631e6652ff""",630606,2024-04-07 19:31:13.615,2024-04-07 19:31:13.820,100,131072,150000000000,150000000000,100000000,150000000000,"""0x975a2f2f68b241078754fd6e4f7ffba717f2ca764d8b3b5b0ad361b123e23ec8""",1,8807856,2024-04-07 19:31:35,"""0x8b5c781398be63ef55abdbcf96a06adcdac5fc8b677e41f9c0b78b900abe2fa4ce18a99c7dff227a737227efc830b7b8""",0,21.385,2,2.38,2,19605912,"""0x2c169dfe5fbba12957bdd0ba47d9cedbfe260ca7""","""0xc662c410c0ecf747543f5ba90660f6abebd9c8c4""",5500000,80,15661490594,15661490594,153769,7060702,150000000000,100000000,15.661,150,0.1
"""0x016b54a90fc0b5bb031edbf82df10e20d73cafebe9f8d880091d3b05b8b0ce51""",630607,2024-04-07 19:31:43.846,2024-04-07 19:31:44.037,100,131072,150000000000,150000000000,100000000,150000000000,"""0xcf0d883cacb248393fadb9208e4dc0bbcd3f8783d9faff0b4f19055bf406c8b4""",1,8807860,2024-04-07 19:32:23,"""0x92166b63ae9d80e172d98345782e6b27935799b0a191a40a7b057798bfecaebe04b4dfd5c5e434d38930a4df7cfb72e2""",0,39.154,4,2.44,2,19605916,"""0x2c169dfe5fbba12957bdd0ba47d9cedbfe260ca7""","""0xc662c410c0ecf747543f5ba90660f6abebd9c8c4""",5500000,92,15816661278,15816661278,165974,7064963,150000000000,100000000,15.817,150,0.1
"""0x013aaf17faa2f3951523b1e157795f7b13565229423389b1a9d7626b8f8a6931""",630608,2024-04-07 19:32:43.980,2024-04-07 19:32:44.277,100,131072,150000000000,150000000000,100000000,150000000000,"""0x74dabe25cee078a1248e0cb2f82d14fdc77f05715884849a28667faa1ca64ed5""",1,8807862,2024-04-07 19:32:47,"""0x876987decf2b1569888b11202fbe9c794f7f6c72eefb7bf6585bd993d024cfb72811e3650d4d98890d432ce7dba90556""",0,3.02,1,2.44,2,19605918,"""0x2c169dfe5fbba12957bdd0ba47d9cedbfe260ca7""","""0xc662c410c0ecf747543f5ba90660f6abebd9c8c4""",5500000,146,16311799555,16311799555,136612,13279368,150000000000,100000000,16.312,150,0.1
"""0x012ce7f2d3977ee3fd49fee6784bdd60528219abaf8e4a32b91cf7654ed084bb""",630609,2024-04-07 19:33:14.684,2024-04-07 19:33:15.128,100,131072,150000000000,150000000000,100000000,150000000000,"""0x4093226e12bf07320edae1217f2283b9d13b83ae144b3af51d831bc31b09fbb1""",1,8807865,2024-04-07 19:33:23,"""0x8a390a20d48c26caa5297f982438245a7104d8dfd175fd5488842c470dd417d65f26c5ce85f30c9da8c1acdf388c8a95""",0,8.316,1,2.42,2,19605921,"""0x2c169dfe5fbba12957bdd0ba47d9cedbfe260ca7""","""0xc662c410c0ecf747543f5ba90660f6abebd9c8c4""",5500000,96,16727753763,16727753763,166202,8070951,150000000000,100000000,16.728,150,0.1


### Stats at a Glance

In [8]:
slot_inclusion_tx_df.select("slot inclusion rate").shape

(1189, 1)

In [9]:
slot_chance_bar_df = (
    slot_inclusion_tx_df.select("hash", "slot inclusion rate")
    .unique()
    .with_columns(
        pl.when(pl.col("slot inclusion rate") == 1)
        .then(True)
        .otherwise(False)
        .alias("1 slot"),
        pl.when(pl.col("slot inclusion rate") == 2)
        .then(True)
        .otherwise(False)
        .alias("2 slots"),
        pl.when(pl.col("slot inclusion rate") >= 3)
        .then(True)
        .otherwise(False)
        .alias("3+ slots"),
    )
    .with_columns(
        pl.col("1 slot").sum(),
        pl.col("2 slots").sum(),
        pl.col("3+ slots").sum(),
        # pl.col('4+ slots').sum()
    )
    .select("1 slot", "2 slots", "3+ slots")[0]
)

In [10]:
slot_chance_bar_chart = slot_chance_bar_df.plot.barh(
    ylabel="number of txs",
    title="Slot Inclusion Breakdown",
    stacked=True,
    width=1000,
    height=400,
)

In [11]:
slot_chance_bar_chart

:Bars   [index,Variable]   (value)

### Slot Inclusion Time Charts

In [12]:
slot_inclusion_line_chart = (
    slot_inclusion_tx_df.drop_nulls()
    .select(
        "slot time",
        "slot inclusion rate",
        "slot inclusion rate (50 blob average)",
        "slot target inclusion rate (2 slots)",
        "submission_count",
    )
    .plot.line(
        x="slot time",
        y=[
            "slot inclusion rate",
            "slot inclusion rate (50 blob average)",
            "slot target inclusion rate (2 slots)",
        ],
        color=["blue", "red", "black"],
        ylabel="Beacon Block Inclusion (block)",
        xlabel="Slot Date Time",
        title="Historical Slot Inclusion",
        width=1000,
        height=400,
        shared_axes=True,
    )
)

### Priority Fee Effectiveness for Faster Slot Inclusion

In [13]:
priority_fee_premium_df = slot_inclusion_tx_df.with_columns(
    (
        (pl.col("max_priority_fee_per_gas_gwei") / pl.col("effective_gas_price_gwei"))
        * 100
    )
    .round(3)
    .alias("priority_fee_bid_percent_premium")
)

In [14]:
priority_fee_premium_chart = (
    (
        priority_fee_premium_df.select(
            "block_number",
            "max_priority_fee_per_gas_gwei",
            "effective_gas_price_gwei",
            "priority_fee_bid_percent_premium",
            "slot inclusion rate",
            "submission_count",
        )
        .unique()
        .sort(by="block_number")
        .with_columns(
            (
                # estimate min block gas by taking the gwei paid minus the priority fee
                pl.col("effective_gas_price_gwei")
                - pl.col("max_priority_fee_per_gas_gwei")
            ).alias("min_block_gas_gwei")
        )
        .with_columns(
            # calculate per tx gas fluctuations
            pl.col("min_block_gas_gwei").diff().abs().alias("gas_fluctuation_gwei")
        )
        .with_columns(
            (pl.col("gas_fluctuation_gwei") / pl.col("min_block_gas_gwei") * 100).alias(
                "gas_fluctuation_percent"
            )
        )
    )
    .group_by("slot inclusion rate")
    .agg(
        pl.col("gas_fluctuation_percent").mean(),
        pl.col("priority_fee_bid_percent_premium").mean(),
    )
    .sort(by="slot inclusion rate")
    .plot.scatter(
        x="slot inclusion rate",
        y="priority_fee_bid_percent_premium",
        ylabel="priority fee bid premium %",
        title="mean priority fee bid premium per slot inclusion rate",
        width=1000,
        height=400,
    )
)

In [15]:
priority_fee_premium_df = (
    priority_fee_premium_df.select(
        "block_number",
        "max_priority_fee_per_gas_gwei",
        "effective_gas_price_gwei",
        "priority_fee_bid_percent_premium",
        "slot inclusion rate",
        "submission_count",
    )
    .unique()
    .sort(by="block_number")
    .with_columns(
        (
            # estimate min block gas by taking the gwei paid minus the priority fee
            pl.col("effective_gas_price_gwei")
            - pl.col("max_priority_fee_per_gas_gwei")
        ).alias("min_block_gas_gwei")
    )
    .with_columns(
        # calculate per tx gas fluctuations
        pl.col("min_block_gas_gwei")
        .diff()
        .abs()
        .alias("gas_fluctuation_gwei")
    )
    .with_columns(
        (pl.col("gas_fluctuation_gwei") / pl.col("min_block_gas_gwei") * 100).alias(
            "gas_fluctuation_percent"
        )
    )
)

In [16]:
volin_chart = priority_fee_premium_df.sort(by="slot inclusion rate").plot.violin(
    y="priority_fee_bid_percent_premium",
    by="slot inclusion rate",
    c="slot inclusion rate",
    width=1000,
    height=400,
)

line_chart_bid_premium = (
    priority_fee_premium_df.group_by("slot inclusion rate")
    .agg(
        pl.col("priority_fee_bid_percent_premium").median(),
        pl.col("effective_gas_price_gwei").mean(),
    )
    .sort(by="slot inclusion rate")
    .rename(
        {
            "priority_fee_bid_percent_premium": "priority fee bid premium (%)",
            "effective_gas_price_gwei": "block gas price (gwei)",
        }
    )
    .plot.line(
        x="slot inclusion rate",
        y=["priority fee bid premium (%)", "block gas price (gwei)"],
        color=["g", "r"],
        ylabel="bid premium (%, gwei)",
        title="gas bid premium probability distributions",
    )
)

In [17]:
violin_line_gas_chart = volin_chart * line_chart_bid_premium

### Assemble Dashboard

In [18]:
dash = pn.Column(
    pn.pane.Markdown(
        """
        # EIP-4844 Slot Inclusion Dashboard

        ## About
        This dashboard shows detailed analytics for blobs and how fast they are included into the next slot as well as the efficiency of using EIP-1559 priority fees
        as a bidding mechanism for faster slot inclusion. As of April 11, 2024, the data is currently restricted to Base posting data over a 5 day period with 
        plans to open up to other rollups soon. This dashboard is made using [Xatu Data](https://github.com/ethpandaops/xatu-data?tab=readme-ov-file) for EL mempool and Beacon chain data and [Hypersync](https://github.com/enviodev/hypersync-client-python) 
        for transaction gas data for the [EIP-4844 data challenge](https://esp.ethereum.foundation/data-challenge-4844).

        ## Summary of Results
        - Blobs have a highly variable slot inclusion rate, with a nontrivial amount of blobs taking 3+ slots to be included.   
        - Higher EIP-1559 priority fees correlate with longer slot inclusion rates.
        - Uncertainty around block gas and priority fee bidding premiums increase with longer slot inclusion rates.
        
        ## Slot Inclusion Methodology
        When a transaction is resubmitted with updated gas parameters, the transaction hash changes. For example take this blob reference hash - 0x01c738cf37c911334c771f2295c060e5bd7d084f347e4334863336724934c59a. 
        On [etherscan](https://etherscan.io/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69) we can see that the transaction was replaced by the user. We can see on Ethernow that the transaction contains 
        the same blob reference hash in both the [original tx](https://www.ethernow.xyz/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69?batchIndex=1) and the [resubmitted tx](https://www.ethernow.xyz/tx/0x5a4094662bd05ff3639a8979927ab527e007a6925387951a9c1b3d2958b13a86?batchIndex=1).
        
        We can measure the total time that a blob hash sat in the mempool by subtracting the original tx was first seen from the slot time, when it eventually is finalized on the beacon chain. 
        In this particular example, the total time that the blob sat in the mempool was not from 18:56:27 to 18:57:11 (4 slots), but really 18:54:29 to 18:57:11 (14 slots)
            """
    ),
    pn.Column(
        pn.pane.Markdown(
            """
            ## Slot Inclusion Breakdown
            This bar chart represents the number of transactions that are being included within 1 slot, 2 slots, and 3+ slots. 
            * 1 slot = fastest possible inclusion time
            * 2 slots = good inclusion time
            * 3+ slots = slow inclusion time
            """
        ),
        slot_chance_bar_chart.opts(axiswise=True),
        pn.pane.Markdown(
            """
        ## Slot Inclusion Rates
        This time-series chartshows the slot inclusion performance over time from when it's first seen in the mempool to when it gets finalized in the beacon block.
        A 50 blob slot inclusion average is taken to smooth out the performance. The target slot inclusion rate is 2. To the right is the overall distribution of the slot inclusion rate.
        While a lot of blobs get finalized on the beacon chain within 1-2 blocks, there are still a non-trivial amount of outliers that can take upwards of 5-10 blocks before finalizing.
            """
        ),
        pn.Row(slot_inclusion_line_chart.opts(axiswise=True)),
        pn.pane.Markdown(
            """
        ## EIP-1559 Priority Fee Premium Correlation with Slot Rates
        The scatterplot shows the priority fee bid premium. There isn't a strong correlation between higher priority fee bid premiums and shorter slot inclusion rates. On the contrary,
        the longer it takes for the blob to be included, the priority fee bid premium starts to increase.
            """
        ),
        pn.Row(priority_fee_premium_chart),
        pn.pane.Markdown(
            """
        ## Slot Inclusion Bidding Unpredictability 
        This major takeaway is that uncertainty around both block gas and priority fee bidding tends to increase the longer the it takes for a blob to be included in a slot. There are three major 
        characteristics in the chart - one violin plot for the probability distribution and two trend lines for priority fee bids and block gas prices.
        * violin plots for each slot inclusion rate, showing the probability distribution of the priority gas bidding premium for each inclusion slot. 
        The higher the slot inclusion rate, the more uncertainty there is around 
        * the green line shows the median trend line for the priority fee bid premium (in %), which tends to drift higher as slot inclusion rate gets higher.
        * the red line shows the mean trend line for the block gas price (in gwei), which drifts upwards was slot inclusion rate gets higher.
            """
        ),
        pn.Row(violin_line_gas_chart),
    ),
)

In [19]:
# dash.show()
dash.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'2d98bcc1-dfce-47a0-929b-4f6987300da8': {'version…